# Praktikum Minggu 3: Apache Spark & PySpark

**Mata Kuliah:** Big Data Analitik  
**Topik:** Apache Spark — RDD, DataFrame, dan Spark SQL  
**Tujuan:** Mahasiswa mampu membuat dan memanipulasi RDD serta DataFrame menggunakan PySpark

---

*Week 3 Lab: Apache Spark & PySpark*  
*Topics covered: SparkSession, RDD operations, DataFrame API, Spark SQL*

In [ ]:
!pip install pyspark

## 1. Import Library & Membuat SparkSession

**SparkSession** adalah titik masuk utama untuk semua fungsionalitas Spark. Sejak Spark 2.0, SparkSession menggantikan SparkContext, SQLContext, dan HiveContext.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import warnings
warnings.filterwarnings('ignore')

# Membuat SparkSession
spark = SparkSession.builder \
    .appName("PraktikumMinggu3") \
    .master("local[*]") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

# Atur level log agar tidak terlalu verbose
spark.sparkContext.setLogLevel("ERROR")

print("SparkSession berhasil dibuat!")
print(f"Versi Spark : {spark.version}")
print(f"App Name   : {spark.sparkContext.appName}")
print(f"Master     : {spark.sparkContext.master}")

## 2. Membuat RDD (Resilient Distributed Dataset)

RDD adalah struktur data fundamental Spark. RDD bersifat:
- **Immutable**: Tidak dapat diubah setelah dibuat
- **Distributed**: Tersebar di beberapa partisi
- **Lazy**: Transformasi tidak langsung dieksekusi
- **Fault-tolerant**: Dapat direkonstruksi dari lineage

In [ ]:
sc = spark.sparkContext

# --- Membuat RDD dari list Python ---
angka = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd_angka = sc.parallelize(angka, numSlices=2)

print("=== Informasi RDD ===")
print(f"Jumlah partisi : {rdd_angka.getNumPartitions()}")

# Operasi dasar
print(f"\ncount()  -> Jumlah elemen : {rdd_angka.count()}")
print(f"first()  -> Elemen pertama: {rdd_angka.first()}")
print(f"take(3)  -> 3 elemen awal : {rdd_angka.take(3)}")
print(f"collect()-> Semua elemen  : {rdd_angka.collect()}")

# Operasi statistik
print(f"\nsum()  = {rdd_angka.sum()}")
print(f"min()  = {rdd_angka.min()}")
print(f"max()  = {rdd_angka.max()}")
print(f"mean() = {rdd_angka.mean()}")

# Transformasi map & filter
rdd_kuadrat = rdd_angka.map(lambda x: x ** 2)
print(f"\nmap(x**2)                   : {rdd_kuadrat.collect()}")

rdd_genap = rdd_angka.filter(lambda x: x % 2 == 0)
print(f"filter(x % 2 == 0)          : {rdd_genap.collect()}")

# reduce
total = rdd_angka.reduce(lambda a, b: a + b)
print(f"\nreduce(a + b) = {total}")

## 3. Membuat DataFrame dengan PySpark

DataFrame adalah abstraksi data tingkat tinggi dengan skema kolom yang terdefinisi. DataFrame menawarkan performa lebih baik dari RDD berkat **Catalyst Optimizer** dan **Tungsten Execution Engine**.

In [ ]:
# --- Membuat DataFrame dari list of tuples dengan skema ---
data_mahasiswa = [
    (1, "Budi Santoso",    "Informatika",        3.85, 2021),
    (2, "Ani Rahayu",      "Sistem Informasi",   3.50, 2021),
    (3, "Citra Dewi",      "Informatika",        3.92, 2022),
    (4, "Dodi Prakoso",    "Sistem Informasi",   3.20, 2022),
    (5, "Eka Putri",       "Informatika",        3.75, 2021),
    (6, "Fajar Nugroho",   "Teknik Komputer",    3.60, 2023),
    (7, "Gita Lestari",    "Informatika",        3.88, 2022),
    (8, "Hendra Wijaya",   "Teknik Komputer",    3.45, 2023),
    (9, "Indah Permata",   "Sistem Informasi",   3.70, 2021),
    (10, "Joko Susilo",    "Teknik Komputer",    3.55, 2022),
]

skema = StructType([
    StructField("id",      IntegerType(), True),
    StructField("nama",    StringType(),  True),
    StructField("jurusan", StringType(),  True),
    StructField("ipk",     FloatType(),   True),
    StructField("angkatan",IntegerType(), True),
])

df_mhs = spark.createDataFrame(data_mahasiswa, schema=skema)

print("=== Tampilan DataFrame ===")
df_mhs.show()

print("\n=== Skema DataFrame ===")
df_mhs.printSchema()

print("\n=== Statistik Deskriptif ===")
df_mhs.describe(["ipk"]).show()

# Select kolom tertentu
print("\n=== Select: nama & IPK ===")
df_mhs.select("nama", "ipk").show(5)

# Filter
print("\n=== Filter: IPK >= 3.75 ===")
df_mhs.filter(F.col("ipk") >= 3.75).show()

# GroupBy & Agregasi
print("\n=== GroupBy Jurusan: Rata-rata IPK ===")
df_mhs.groupBy("jurusan") \
    .agg(
        F.count("*").alias("jumlah_mhs"),
        F.round(F.avg("ipk"), 3).alias("rata_ipk"),
        F.max("ipk").alias("ipk_tertinggi")
    ) \
    .orderBy(F.desc("rata_ipk")) \
    .show()

## 4. Spark SQL

Spark SQL memungkinkan kita menjalankan kueri SQL standar pada DataFrame yang didaftarkan sebagai **temporary view**.

In [ ]:
# Daftarkan DataFrame sebagai temporary view
df_mhs.createOrReplaceTempView("mahasiswa")

print("=== Kueri SQL 1: Semua Mahasiswa ===")
spark.sql("SELECT * FROM mahasiswa ORDER BY ipk DESC").show()

print("\n=== Kueri SQL 2: Rata-rata IPK per Jurusan ===")
spark.sql("""
    SELECT
        jurusan,
        COUNT(*) AS jumlah_mahasiswa,
        ROUND(AVG(ipk), 3) AS rata_rata_ipk,
        MAX(ipk) AS ipk_max,
        MIN(ipk) AS ipk_min
    FROM mahasiswa
    GROUP BY jurusan
    ORDER BY rata_rata_ipk DESC
""").show()

print("\n=== Kueri SQL 3: Mahasiswa IPK > 3.7 per Angkatan ===")
spark.sql("""
    SELECT
        angkatan,
        COUNT(*) AS mhs_berprestasi
    FROM mahasiswa
    WHERE ipk > 3.7
    GROUP BY angkatan
    ORDER BY angkatan
""").show()

print("\n=== Kueri SQL 4: Subquery — Mahasiswa di Atas Rata-rata ===")
spark.sql("""
    SELECT nama, jurusan, ipk
    FROM mahasiswa
    WHERE ipk > (SELECT AVG(ipk) FROM mahasiswa)
    ORDER BY ipk DESC
""").show()

## 5. Transformasi & Aksi pada RDD

Memahami perbedaan **transformasi** (lazy, menghasilkan RDD baru) dan **aksi** (eager, memicu eksekusi dan mengembalikan nilai).

In [ ]:
print("========== TRANSFORMASI (Lazy) ==========")

kalimat = [
    "big data analytics dengan spark",
    "apache spark adalah framework cepat",
    "rdd dataframe dan spark sql"
]
rdd_kalimat = sc.parallelize(kalimat)

# map: ubah setiap kalimat menjadi uppercase
rdd_upper = rdd_kalimat.map(lambda s: s.upper())
print("\nmap (uppercase):")
print(rdd_upper.collect())

# flatMap: pecah kalimat menjadi kata-kata
rdd_kata = rdd_kalimat.flatMap(lambda s: s.split(" "))
print("\nflatMap (split kata):")
print(rdd_kata.collect())

# filter: ambil kata dengan panjang > 4 karakter
rdd_kata_panjang = rdd_kata.filter(lambda w: len(w) > 4)
print("\nfilter (panjang > 4 karakter):")
print(rdd_kata_panjang.collect())

# distinct: hapus duplikat
rdd_unik = rdd_kata.distinct()
print("\ndistinct (kata unik):")
print(sorted(rdd_unik.collect()))

# union: gabungkan dua RDD
rdd_a = sc.parallelize(["spark", "hadoop", "kafka"])
rdd_b = sc.parallelize(["kafka", "flink", "spark"])
rdd_union = rdd_a.union(rdd_b)
print("\nunion:")
print(rdd_union.collect())

# sortBy: urutkan kata berdasarkan panjang
rdd_sort = rdd_kata_panjang.sortBy(lambda w: len(w), ascending=False)
print("\nsortBy (terpanjang):")
print(rdd_sort.collect())

print("\n========== AKSI (Eager) ==========")
rdd_nums = sc.parallelize([3, 7, 1, 9, 2, 5, 8, 4, 6, 10])

print(f"count()         : {rdd_nums.count()}")
print(f"collect()       : {rdd_nums.collect()}")
print(f"first()         : {rdd_nums.first()}")
print(f"take(4)         : {rdd_nums.take(4)}")
print(f"top(3)          : {rdd_nums.top(3)}")
print(f"reduce(max)     : {rdd_nums.reduce(lambda a, b: a if a > b else b)}")
print(f"sum()           : {rdd_nums.sum()}")

# countByValue
rdd_warna = sc.parallelize(["merah","biru","merah","hijau","biru","merah"])
print(f"\ncountByValue()  : {dict(rdd_warna.countByValue())}")

## Tugas Praktikum

Kerjakan soal-soal berikut secara mandiri:

**Soal 1 — RDD Dasar:**  
Buatlah RDD dari daftar 20 bilangan acak (antara 1–100). Hitung jumlah bilangan prima dalam RDD tersebut menggunakan transformasi `filter`.

**Soal 2 — Word Count:**  
Implementasikan program **Word Count** klasik menggunakan RDD. Gunakan teks berikut: `"the quick brown fox jumps over the lazy dog the dog barked at the fox"`. Tampilkan 5 kata yang paling sering muncul.

**Soal 3 — DataFrame Penjualan:**  
Buat DataFrame dengan kolom `produk`, `kategori`, `harga`, `jumlah_terjual`. Isi dengan minimal 15 baris data. Lakukan:
- Hitung total pendapatan per kategori (harga × jumlah_terjual)
- Temukan produk dengan pendapatan tertinggi
- Filter produk dengan harga di atas rata-rata

**Soal 4 — Spark SQL Lanjutan:**  
Dengan DataFrame mahasiswa yang sudah dibuat, tulis kueri Spark SQL untuk:
- Menampilkan peringkat IPK mahasiswa per jurusan menggunakan fungsi window `RANK()`
- Menampilkan nama mahasiswa dengan IPK tertinggi di setiap angkatan

**Soal 5 — Analisis Teks:**  
Buat RDD dari file teks (gunakan `sc.parallelize` dengan daftar kalimat minimal 10 baris). Hitung:
- Rata-rata panjang kata dalam setiap kalimat
- Jumlah kalimat yang mengandung kata "data"
- Daftar kata unik yang muncul lebih dari satu kali

In [ ]:
# Hentikan SparkSession setelah selesai
spark.stop()
print("SparkSession dihentikan. Praktikum selesai!")